# Imports 

In [1]:
import pickle, gzip, time, torch, os, matplotlib as mpl, shutil, numpy as np
from pathlib import Path
from torch import tensor 
from fastcore.test import test_close

# important! so I can compare my work
torch.manual_seed(42)

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'

with gzip.open(path_gz, 'rb') as f: 
   ((x_train,y_train),(x_valid,y_valid),_) = pickle.load(f, encoding = 'latin-1')

x_train,y_train,x_valid,y_valid = map(tensor,(x_train,y_train,x_valid,y_valid))

# Basic Architecture

We will use the basic shape of the input to define the architecture of our layers. 

Following the lesson, we'll create 2 linear layers with a activation layer (reLu) in between. 

The first layer with have 784 neurons to match the 784 inputs. It will have 50 hidden neurons. The second layer will have 50 inputs and 1 output.

In [2]:
y_train.shape,x_train.shape

(torch.Size([50000]), torch.Size([50000, 784]))

In [3]:
n,m = x_train.shape
c = y_train.max()+1
nh = 50

In [4]:
w1 = torch.randn(784,50)
b1 = torch.randn(50)
w2 = torch.randn(50,1)
b2 = torch.randn(1)

In [5]:
w1 = torch.randn(m,nh)
b1 = torch.randn(nh)
w2 = torch.randn(nh,1)
b2 = torch.randn(1)

The above was my first implementation. Not wrong, but I notice that jeremy initialized the biases with `torch.zeros(*size)` rather than `torch.randn(*size)`. Does that make any difference? I'll keep mine for now

Jeremy implemented the model in three ways:

1. Functions
2. Classes
3. Classes with inheritance

Let's begin by creating functions for the linear operation and for reLu

In [6]:
def linear(xb,w,b): return xb@w+b

In [7]:
def relu(inp): return inp.clamp_min(0)

Seems okay, why don't we try it out?

In [8]:
inter = linear(x_train,w1,b1)
inter

tensor([[  0.03, -11.21,   1.34,  ...,  11.02,   3.02,  -3.79],
        [ -3.41, -12.25,  -7.79,  ...,  31.63,   9.03, -11.11],
        [  5.08,  -0.43,  -6.60,  ...,   5.51,  -5.56,  -7.53],
        ...,
        [ -2.86, -12.46,   0.61,  ...,  14.29,  -5.90,  -6.44],
        [  7.43, -19.30, -10.29,  ...,   4.37,  -0.15,  -7.13],
        [  9.10, -18.73,   6.79,  ...,   0.36, -16.52, -14.49]])

In [9]:
preds = relu(inter)
preds

tensor([[ 0.03,  0.00,  1.34,  ..., 11.02,  3.02,  0.00],
        [ 0.00,  0.00,  0.00,  ..., 31.63,  9.03,  0.00],
        [ 5.08,  0.00,  0.00,  ...,  5.51,  0.00,  0.00],
        ...,
        [ 0.00,  0.00,  0.61,  ..., 14.29,  0.00,  0.00],
        [ 7.43,  0.00,  0.00,  ...,  4.37,  0.00,  0.00],
        [ 9.10,  0.00,  6.79,  ...,  0.36,  0.00,  0.00]])

Looks like it works! But I might want to do another linear layer. Where do I fit that in? I just call linear again with different args

In [10]:
preds = linear(preds,w2,b2)

In [11]:
preds.shape

torch.Size([50000, 1])

# MSE

Cool! That works. We get a tensor of the right shape, but we don't just want predictions, we need a way to compute the loss. Let's write a function to compute the loss. 

Again, following Jeremy, I will use mse for the loss function even though that is totally inappopriate for this kind of prediction. After we've got backpropagation working properly, we'll move onto a better loss function like cross entropy. 

In [12]:
def mse(output, targs):
    return (output[:,0]-targs).pow(2).mean()

In [13]:
loss = mse(preds,y_train)
loss

tensor(1590.84)

This answer is different from Jeremy's. I've made a few deviations that might explain why. 
1. I didn't convert my y_train to float. 

In [14]:
y_train.float()

tensor([5., 0., 4.,  ..., 8., 4., 8.])

In [15]:
loss = mse(preds,y_train)
loss

tensor(1590.84)

Same thing! I'm going to try restarting the kernel. Didn't work. I got the same answer. Hmmm. Well, I'm not sure I have any other reason to suspect this is wrong. So I'll roll with it. 

The other thing I did differently was that I didn't roll my layers into a model. I'll follow him on that. 

In [16]:
def model(xb): 
    l1 = linear(xb,w1,b1)
    l2 = relu(l1)
    return linear(l2,w2,b2)

In [17]:
preds2 = model(x_train)

In [18]:
preds2

tensor([[   -43.42],
        [   -99.04],
        [    16.26],
        ...,
        [   -41.02],
        [   -11.81],
        [    -0.01]])

In [19]:
preds

tensor([[   -43.42],
        [   -99.04],
        [    16.26],
        ...,
        [   -41.02],
        [   -11.81],
        [    -0.01]])

## Quick Digression - Loss Verification
How can I be sure my loss is correct? Compute it manually on a small set and compare.

In [20]:
x_tst = x_train[:5]
y_tst = y_train[:5]
x_tst.shape,y_tst.shape

(torch.Size([5, 784]), torch.Size([5]))

In [21]:
y_tst

tensor([5, 0, 4, 1, 9])

In [22]:
preds3 = model(x_tst)
preds3

tensor([[-43.42],
        [-99.04],
        [ 16.26],
        [ 15.32],
        [-40.14]])

In [23]:
calc_loss = mse(preds3,y_tst)

In [24]:
calc_loss

tensor(2984.56)

In [25]:
preds[:,0]

tensor([   -43.42,    -99.04,     16.26,  ...,    -41.02,    -11.81,     -0.01])

In [26]:
b = preds3[:,0]-y_tst
b

tensor([-48.42, -99.04,  12.26,  14.32, -49.14])

In [27]:
b.pow(2)

tensor([2344.19, 9808.50,  150.37,  205.19, 2414.57])

In [28]:
b.pow(2).mean()

tensor(2984.56)

In [29]:
-48.42*-48.42

2344.4964

Looks good

# Gradients and Backprop

In [30]:
def lin_grad(inp, out, w, b): 
    inp.g = out.g @ w.t()
    w.g = inp.t()@out.g
    #w.g = inp.T@out.g 
    #-- this method doesn't work for all cases. It runs into trouble at the highest layers where the output and input tensors become rank 1
    # in that case, there is nothing to transpose and the axes will be different sizes. I'm confused. Why doesn't the above have the same problem?
    # This method, by contrast, will creates unit axes that allow us to broadcast operations 
    #import pdb; pdb.set_trace()
    w.g = (inp.unsqueeze(-1)*out.g.unsqueeze(1)).sum(0)
    # actually, the above methods work-- I just need to remember to do a matrix multiply '@' instead of elementwise opp '*'
    b.g = out.g.sum(0)

Why do we sum for gradients of w and b? We want to integrate over the relative contributions for each input. 

To put it another way, each weight/biases has some impact on the loss relative to its input. I.e., the derivative of the loss with respect to w1 is x for this input and is y for input 2 and z for input 3, etc. We want w1's total derivative, one equation, so we need to sum that contribution over all the inputs to get the true contribution of w1 (and all the other weights). 

Each input, on the other hand, is not seperated across weights. The weights are aligned to the input, that's why the matrix matplication works. so there's no need to sum. Each input already gets its derivative handled. 

In [31]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = linear(inp, w1, b1)
    l2 = relu(l1)
    out = linear(l2, w2, b2)
    diff = out[:,0]-targ
    loss = diff.pow(2).mean()
    
    # backward pass:
    out.g = 2.*diff[:,None] / inp.shape[0]
    lin_grad(l2, out, w2, b2)
    l1.g = (l1>0).float() * l2.g
    lin_grad(inp, l1, w1, b1)

In [32]:
forward_and_backward(x_train, y_train)

In [91]:
def get_grad(x): return x.g.clone()

chks = w1,w2,b1,b2,x_train
grads = w1g, w2g,b1g,b2g,ig = tuple(map(get_grad,chks))

In [92]:
def mkgrad(x): return x.clone().requires_grad_(True)

ptgrads = w12,w22,b12,b22,xt2 = tuple(map(mkgrad,chks))

In [96]:
def forward(inp, targ): 
    l1 = linear(inp, w12, b12)
    l2 = relu(l1) 
    out = linear(l2, w22, b22)
    return mse(out, targ)

In [97]:
loss = forward(xt2, y_train)
loss.backward()

In [102]:
for a,b in zip(grads, ptgrads): test_close(a,b.grad, eps=0.01)

Ok! Where do do I go from here? If I remember correctly -- Jeremy wrote out each of his layers as classes so that they could hold information about their gradients. There was some discussion of chain rule as well. I remember this being confusing for me, so I will solider on.

In [33]:
class Mse(): 
    def __call__(self, inp, targs):
        self.inp,self.targs = inp,targs
        self.out = (inp[:,0]-targs).pow(2).mean()
        return self.out
        
    def backward(self):
        self.inp.g = 2*((output[:,0]-targs))/output.shape[0]

In [34]:
class Relu(): 
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0)
        return self.out
        
    def backward(self):
        self.inp.g =(self.inp>0).float()*self.out.g 

In [35]:
class Lin():
    def __init__(self,w,b):
        self.w = w
        self.b = b 
        
    def __call__(self, inp): 
        self.inp = inp
        self.out = inp@self.w+self.b 
        return self.out
        
    def backward(self):
        self.w.g = inp.t()@self.out.g
        self.inp.g = self.out.g@self.w.T
        self.b.g = self.out.g.sum(0)

In [36]:
class Model():
    def __init__(self,w1, w2,b1,b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()

    def __call__(self, x, targ):
        
        # forward pass
        for l in self.layers: 
            x = l(x)
        
        # init backward pass with loss calculation
        return self.loss(x, targ)
    
    def backward(self):
        
        # continue backward pass
        self.loss.backward()
        
        # call backward()-->.bwd() on each layer
        for l in reversed(self.layers): 
            l.backward()

In [37]:
model = Model(w1,w2,b1,b2)

In [38]:
loss = model(x_train, y_train)

In [39]:
loss

tensor(1590.84)

## Module.forward()

In [40]:
# Base Class for Inheritance
class Module():    
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out

    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)
    def bwd(self): raise Exception('not implemented')

In [41]:
class Relu(Module):
    
    def forward(self, inp):
        return inp.clamp_min(0)
    
    def bwd(self, out, inp):
        inp.g = (inp>0).float() * out.g

In [42]:
class Lin(Module): 
    def __init__(self, w,b): self.w, self.b = w,b 
    def forward(self, inp): return inp @ self.w + self.b
    def bwd(self, out, inp):
        inp.g = self.out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [43]:
class Mse(Module): 
    def forward(self, inp, targ): return (inp[:,0]-targ).pow(2).mean(0)
    def bwd(self,out,inp,targ): inp.g = 2*(inp[:,0]-targ).unsqueeze(-1)/inp.shape[0]

In [44]:
model = Model(w1,w2,b1,b2)

In [45]:
loss = model(x_train,y_train)

In [46]:
model.backward()

## Autograd

In [70]:
from torch import nn
import torch.nn.functional as F

In [81]:
y_train = y_train.float()
x_train = x_train.float()

In [82]:
class Linear(nn.Module): 
    def __init__(self, n_in, n_out):
        super().__init__()
        self.w = torch.randn(n_in,n_out).requires_grad_()
        self.b = torch.zeros(n_out).requires_grad_()
    def forward(self,inp): return inp@self.w+self.b
        

In [83]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [Linear(n_in,nh), nn.ReLU(), Linear(nh,n_out)]
        
    def __call__(self, x, targs):
        
        # forward pass
        for l in self.layers: 
            x = l(x) 
         
        return F.mse_loss(x, targs[:,None]) 

In [84]:
model = Model(x_train.shape[1],50,1)

In [85]:
loss = model(x_train, y_train)

In [86]:
loss.backward()

In [87]:
l0 = model.layers[0]
l0.b.grad

tensor([-16.35, -61.78,  14.58,   5.10,  69.91, -78.21,  50.94, 276.71, 214.24,  16.58, 228.04,  48.73, -26.02, 197.18,
        129.62, -26.05, -49.19, 127.78, -28.31,   2.96, -22.60, -23.90, -63.58,  -0.47,  35.05, -41.11,  14.80,  26.05,
        -11.11, -68.51, -13.66, 264.30,  24.00,  -4.00,  17.84, 117.48, -15.01, -50.04, 116.87,  40.59,  28.73, -91.12,
         21.37,  32.92, -40.96, -24.52,  37.49, -97.57, 137.62, 127.02])